In [2]:
import pandas as pd
df = pd.read_csv(r"C:\Users\es422\Documents\xentity\BLM\MLRS\Data\Snapshots\MLRS\2025-03-02_MLRS_Full\2025-03-02\consolidated_tables\CR_FULL_BLM_CASE.csv", sep='|', low_memory=False)
print(f"Number of rows in file: {df.shape[0]}")

C:\Users\es422\AppData\Local\Temp\ipykernel_10492\2124138203.py:2: DtypeWarning: Columns (10,13,17,19,21,24,25,27,30,38,43,53,54,55,57,58,70,73,74,76,77,78,79,80,84,89,92,93,94,95,96,97,100,101,112,115,116) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r"C:\Users\es422\Documents\xentity\BLM\MLRS\Data\Snapshots\MLRS\2025-03-02_MLRS_Full\2025-03-02\consolidated_tables\CR_FULL_BLM_CASE.csv", sep='|')


Number of rows in file: 6187847


In [3]:
import pandas as pd

df = pd.read_parquet(r"C:\Users\es422\Documents\xentity\BLM\MLRS\Data\Snapshots\MLRS\2025-03-02_MLRS_Full\2025-03-02\consolidated_tables\CR_FULL_BLM_CASE.parquet")
print(f"Number of rows in file: {df.shape[0]}")

Number of rows in file: 6187847


In [4]:
import os
import sys

# Set Hadoop environment variables
os.environ['HADOOP_HOME'] = r'C:\hadoop'
os.environ['PATH'] = os.environ['PATH'] + r';C:\hadoop\bin'

# Create SparkSession with datetime rebase configuration
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("LocalSpark") \
    .config("spark.driver.memory", "16g") \
    .config("spark.sql.parquet.datetimeRebaseModeInRead", "LEGACY") \
    .config("spark.sql.parquet.datetimeRebaseModeInWrite", "LEGACY") \
    .config("spark.sql.parquet.int96RebaseModeInWrite", "LEGACY") \
    .getOrCreate()

df = spark.read.parquet(r"C:\Users\es422\Documents\xentity\BLM\MLRS\Data\Snapshots\MLRS\2025-03-02_MLRS_Full\2025-03-02\consolidated_tables\CR_FULL_BLM_CASE.parquet")
print(f"Number of rows in file: {df.count()}")

Number of rows in file: 6187847


In [5]:
import dask.dataframe as dd

df = dd.read_parquet(r"C:\Users\es422\Documents\xentity\BLM\MLRS\Data\Snapshots\MLRS\2025-03-02_MLRS_Full\2025-03-02\consolidated_tables\CR_FULL_BLM_CASE.parquet")
print(f"Number of rows: {len(df)}")

c:\Users\es422\anaconda3\envs\geo\Lib\site-packages\dask\dataframe\_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 12.0.1 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


Number of rows: 6187847


In [6]:
import dask.dataframe as dd

df = dd.read_csv(
    r"C:\Users\es422\Documents\xentity\BLM\MLRS\Data\Snapshots\MLRS\2025-03-02_MLRS_Full\2025-03-02\consolidated_tables\CR_FULL_BLM_CASE.csv",
    sep='|',
    low_memory=False,  
    blocksize='128MB'
)
print(f"Number of rows: {len(df)}")

ValueError: Mismatched dtypes found in `pd.read_csv`/`pd.read_table`.

+--------+--------+----------+
| Column | Found  | Expected |
+--------+--------+----------+
| 101    | object | float64  |
| 112    | object | float64  |
| 21     | object | float64  |
| 38     | object | float64  |
| 54     | object | float64  |
| 57     | object | float64  |
| 58     | object | float64  |
| 77     | object | float64  |
| 96     | object | float64  |
| 97     | object | float64  |
+--------+--------+----------+

The following columns also raised exceptions on conversion:

- 101
  ValueError("could not convert string to float: 'ALL LEASABLE MINERALS;ALL MINERALS'")
- 112
  ValueError("could not convert string to float: '001t000000ht0E7AAI'")
- 21
  ValueError("could not convert string to float: 'PT'")
- 38
  ValueError("could not convert string to float: 'a023d00000GZEZkAAP'")
- 54
  ValueError("could not convert string to float: 'To Be Defined'")
- 57
  ValueError("could not convert string to floa

In [ ]:
df = spark.read.parquet(r"C:\Users\es422\Documents\xentity\BLM\MLRS\Data\Snapshots\NLSDB\2025_03-03_NLSDB.gdb\CaseLands_03032025_1330.parquet")
df.columns

In [5]:
joined_df = gdf.join(df, gdf["SF_ID_nlsdb"] == df["ID_sf"], "left")


In [6]:
joined_df = joined_df.filter(joined_df["CSE_NR_nlsdb"] != joined_df["SERIAL_NUMBER__C_sf"])

In [7]:
joined_df.count()

2

In [7]:
joined_df.write.mode("overwrite").parquet("nlsdb_joind_blm_case.parquet")

In [8]:
joined_df = spark.read.parquet("nlsdb_joind_blm_case.parquet")
import geopandas as gpd
from shapely import wkb
import pandas as pd

# Convert the Spark DataFrame sample to pandas
pdf = joined_df.limit(50).toPandas()
# pdf[['Shape_nlsdb']]
# Convert the binary array back to bytes, then to shapely geometry
pdf['geometry'] = pdf['Shape_nlsdb'].apply(lambda x: wkb.loads(bytes(x)) if x is not None else None)

# gpd.GeoDataFrame(pdf, geometry='geometry')[44:45].plot()

# # Convert the WKB geometry column back to shapely geometry
# # The geometry column might be named 'SHAPE_nlsdb' or 'geometry_nlsdb' - adjust if needed
# pdf['geometry'] = pdf['Shape_nlsdb'].apply(lambda x: wkb.loads(x) if x else None)

# # Create a GeoDataFrame
# gdf = gpd.GeoDataFrame(pdf, geometry='geometry')

# # Create a simple plot
# ax = gdf.plot(figsize=(10, 10))
# ax.set_title('First 5 Records')

In [13]:
joined_df.limit(5).toPandas()[['CSE_NR_nlsdb','SERIAL_NUMBER__C_sf']]

,CSE_NR_nlsdb,SERIAL_NUMBER__C_sf
0,WYWY106718415,WYWY106344735
1,NDMT105823539,NDMT105485224


In [46]:
gdf.limit(5).filter(gdf['SF_ID_nlsdb']=='a023d00000FghyTAAR').toPandas()

,OBJECTID_nlsdb,CSE_NR_nlsdb,LEG_CSE_NR_nlsdb,REC_TYPE_CSE_GRP_nlsdb,BLM_PROD_nlsdb,CSE_TYPE_NR_nlsdb,CSE_LND_STATUS_nlsdb,CSE_LND_STATUS_DT_nlsdb,CSE_LND_NR_nlsdb,US_RIGHTS_nlsdb,...,SF_ID_nlsdb,SF_CL_ID_nlsdb,ID_nlsdb,AGG_CLS_ID_nlsdb,QLTY_CK_nlsdb,Created_nlsdb,Modified_nlsdb,Shape_Length_nlsdb,Shape_Area_nlsdb,Shape_nlsdb


In [56]:
gdf.select('SF_ID_nlsdb').filter(gdf['SF_ID_nlsdb']).limit(5).toPandas()#[['SF_ID_nlsdb']]

,SF_ID_nlsdb
